In [139]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

In [140]:
df = pd.read_parquet(os.path.join("data","cnb_vyber.parquet"))
df = df.explode('008')
df['rok'] = df['008'].apply(lambda x: najdi_rok(x))
df['stran'] = df['300_a'].apply(lambda x: pocet_stran(x))

In [141]:
df = df.dropna(subset=['245_a','100_a','rok'])

In [142]:
df = df[df['stran'] >= 30]

In [ ]:
df = df.explode('260_b').explode('264_b')

In [ ]:
def dvoje_nakladatelstvi(sedesat, sedesatctyri):
    if sedesat != None:
        return sedesat.replace(',','').replace(":",'').strip()
    elif sedesatctyri != None:
        return sedesatctyri.replace(',','').replace(":",'').strip()
    else:
        return None

In [ ]:
df['nakladatel'] = df.apply(lambda row: dvoje_nakladatelstvi(row['260_b'], row['264_b']), axis=1)

In [ ]:
df[(df['rok'] == 2021) & (df['nakladatel'].isnull())]

In [ ]:
df[df['nakladatel'].notnull()].groupby('rok').size() / df.groupby('rok').size()

In [ ]:
df['nakladatel'].sample(50)

In [ ]:
df[df['100_7'].astype(str).str.contains('jx20100301005',na=False)]

In [ ]:
df[df['nakladatel'] == 'Listen'].sort_values(by='rok')

In [ ]:
df[df['nakladatel'] == 'Paseka'].sort_values(by='rok')

## Nejproduktivnější nakladatelství

In [ ]:
df.groupby('nakladatel').size().nlargest(60)

In [ ]:
df[df['rok'] > 2013].groupby('nakladatel').size().nlargest(60)

## Počet jednotlivých nakladatelství

In [ ]:
df.groupby('rok')['nakladatel'].nunique().plot.bar(figsize=(25,5))

In [ ]:
df.groupby('rok')['nakladatel'].nunique().nlargest()

In [ ]:
df[df['rok'] == 1953].groupby('nakladatel').size().nlargest(15)

## Počet nových nakladatelství

In [ ]:
pd.DataFrame(df.groupby('nakladatel')['rok'].min()).groupby('rok').size().plot.bar(figsize=(25,5))

## Počet končících nakladatelství

In [ ]:
pd.DataFrame(df.groupby('nakladatel')['rok'].max()).groupby('rok').size().plot.bar(figsize=(25,5))

In [ ]:
(pd.DataFrame(df.groupby('nakladatel')['rok'].max()).groupby('rok').size() / df.groupby('rok')['nakladatel'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
(pd.DataFrame(df.groupby('nakladatel')['rok'].max()).groupby('rok').size() / df.groupby('rok')['nakladatel'].nunique()).nlargest(10)

In [ ]:
pd.DataFrame(df[df['rok'] < 1990].groupby('nakladatel')['rok'].max()).groupby('rok').size().nlargest(5)

In [ ]:
pd.DataFrame(df.groupby('nakladatel')['rok'].max()).groupby('rok').size()

## Grafy

In [ ]:
import altair as alt

In [ ]:
nakladatelstvi_s_tremi_knihami = df.groupby("nakladatel").size()
nakladatelstvi_s_tremi_knihami = nakladatelstvi_s_tremi_knihami[nakladatelstvi_s_tremi_knihami >= 3].index.to_list()
df_aspon_tri = df[df['nakladatel'].isin(nakladatelstvi_s_tremi_knihami)]

In [ ]:
slov = {
    'celkem':'Nakladatelství celkem',
    'nova':'Nová nakladatelství',
    'koncici':'Nakladatelství, která vydala svoji (k roku 2025) poslední knihu'
}

jednotliva = pd.DataFrame(df_aspon_tri.groupby('rok')['nakladatel'].nunique()).reset_index().rename(columns={"nakladatel": slov['celkem']})
nova = pd.DataFrame(df_aspon_tri.groupby('nakladatel')['rok'].min()).groupby('rok').size().reset_index().rename(columns={0: slov['nova']})
koncici = pd.DataFrame(df_aspon_tri.groupby('nakladatel')['rok'].max()).groupby('rok').size().reset_index().rename(columns={0: slov['koncici']})
do_grafu = jednotliva.merge(nova, on="rok", how="left")
do_grafu = do_grafu.merge(koncici, on="rok", how="left")
do_grafu = do_grafu.melt(id_vars="rok")
do_grafu = do_grafu[do_grafu['rok'] >= 1900]
do_grafu = do_grafu[((do_grafu['rok'] <= 2023) & (do_grafu['variable'].isin([slov['nova'], slov['celkem']])) | ((do_grafu['rok'] <= 2022) & (do_grafu['variable'] == slov['koncici'])))]
do_grafu['rok'] = pd.to_datetime(do_grafu['rok'].apply(lambda x: f"{int(x)}-01-01"))
do_grafu

In [ ]:
alt.Chart(do_grafu).mark_line().encode(
    x='rok:T',
    y='value:Q',
    color='variable:N',
)

In [ ]:
from textwrap import wrap

In [ ]:
titulek='Kolik různých nakladatelství vydávalo knihy'
podtitulek="Nejhůř poznamenaly pestrost nakladatelské scény nacistická okupace a komunistický režim. Do grafu se promítají pouze nakladatelství tří a více knih."
podtitulek=wrap(podtitulek, 60)

In [ ]:
# First, let's create a condition for the opacity
# We need to transform the data to add an opacity column for the "Končící" row

# Create a copy of the data
import pandas as pd
import altair as alt

# Assuming do_grafu is your original dataframe
fading_data = do_grafu.copy()

# Add opacity column based on year and variable
def calculate_opacity(row):
    if row['variable'] == slov['koncici']:
        # For years up to 2010, opacity is 1 (fully visible)
        if row['rok'].year <= 2000:
            return 1.0
        # For years between 2010 and 2024, opacity gradually decreases
        elif row['rok'].year <= 2024:
            # Linear interpolation from 1.0 at 2010 to 0.0 at 2024
            return 1.0 - ((row['rok'].year - 2000) / 50.0)
        # For years after 2024, opacity is 0 (fully transparent)
        else:
            return 0.0
    # For other variables, opacity is always 1 (fully visible)
    else:
        return 1.0

# Apply the function to create a new 'opacity' column
fading_data['opacity'] = fading_data.apply(calculate_opacity, axis=1)

# Now create the modified chart using the new opacity column
graf_nakl = alt.Chart(
    fading_data, 
    title=alt.Title(
        titulek,
        subtitle=['Nejpestřejší byla nakladatelská scéna v 90. letech a za první republiky.',
                  'Vývoj končících nakladatelství začíná být po roce 2000 zkreslený (vzhůru)',
                  'pauzujícími nakladateli. Grafy zobrazují pouze nakladatele alespoň tří knih.']
    )
).mark_bar(width=2).encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('value:Q', 
          axis=alt.Axis(domainOpacity=0, tickCount=2, tickColor='#DCDDD6', orient='right', labelExpr='replace(datum.label, ",000", " tis.")'), 
          title=None
    ),
    alt.Color("variable:N", 
              scale=alt.Scale(range=['#D55F50', '#ECD410', '#9CA545']),
              title=None, 
              legend=None
    ),
    # Apply opacity based on the new column
    opacity=alt.Opacity('opacity:Q',title=None,legend=None),
    row=alt.Row(
        "variable:N", 
        title=None, 
        spacing=15, 
        header=alt.Header(
            labelAngle=0, 
            labelAlign='left', 
            labelAnchor='start', 
            labelFontWeight=400, 
            labelFont='Asap', 
            labelOrient="top",
            labelPadding=0
        ),
        sort=[slov['celkem'],slov['nova'],slov['koncici']],
    )
).properties(
    width=kredity['sirka'], 
    height=kredity['vyska_nizkych'], 
    autosize={'type': 'fit', 'contains': 'padding'}
).resolve_axis(
    x='independent'
).resolve_scale(
    y='shared'
).configure_view(
    stroke='transparent'
).configure_axisX(
    grid=False, 
    domain=False
)

graf_nakl

In [ ]:
me_to_neurazi(graf_nakl, soubor="01_nakladatelstvi_pocty", kredity=kredity['default'])

In [ ]:
import polars as pl

### Podíl největšího

In [ ]:
nakl = df[df['stran'] >= 30].groupby(["nakladatel","rok"]).size().reset_index().rename(columns={0: 'knih'})
knih_za_rok = nakl.groupby("rok")["knih"].sum().reset_index().rename(columns={'knih': 'celkem'})
nakl = nakl.merge(knih_za_rok, on="rok", how="left")
nakl['podíl'] = nakl['knih'] / nakl['celkem']
nakl

In [ ]:
str(nakl.sample(8))

In [ ]:
nakl.sort_values(by='podíl', ascending=False)

In [ ]:
samonaklad = ['nákladem vlastním','nákl. vlast.','nákl. vl.']

In [ ]:
nakl

In [ ]:
result = nakl[~nakl['nakladatel'].isin(samonaklad)].sort_values(['rok', 'podíl'], ascending=[True, False]).drop_duplicates('rok')
print(result)
result = result[result['rok'].isin(range(1900,2024))]
result['rok'] = pd.to_datetime(result['rok'].apply(lambda x: f"{int(x)}-01-01"))
result = result.replace({'IŽ': 'Železný','Ivo Železný':'Železný','Euromedia Group':'Euromedia'})
nejcastejsi = result.groupby('nakladatel').size() 
print(nejcastejsi.nlargest(15))
nejcastejsi = nejcastejsi[nejcastejsi >=4 ].index.to_list()
print(nejcastejsi)
nejcastejsi=['SPN','MOBA','Železný','Euromedia','Orbis']
nenahrazeni = {x : x for x in nejcastejsi}
print(nenahrazeni)
result = pl.from_pandas(result).with_columns(pl.col('nakladatel').replace_strict(nenahrazeni, default='jiné')).to_pandas()
result

In [ ]:
result['nakladatel'].nunique()

In [ ]:
razeni = result['nakladatel'].drop_duplicates().to_list()

In [ ]:
titulek2 = "Podíl největšího nakladatelství na vydaných knihách"
podtitulek2 = wrap("Za první republiky vydávali nejvíce knih střídavě J. Otto, B. Kočí, Jos. R. Vilímek a Fr. Švejda, často se také na první příčce objevoval samonáklad. Monopol Státního pedagogického nakladatelství za socialismu krátkodobě narušovalo Státní nakladatelství technické knihy. Po revoluci se krátkodobě prosadila nakladatelství Harlequin, BB art, Grada a univerzitní nakladatelství ČVUT a UPOL.",70)

In [ ]:
graf_nejvesi = alt.Chart(result, title={'text': titulek2}).mark_bar(width=2).encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('podíl:Q', 
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', orient='right', labelExpr="datum.label * 100 + ' %'"), 
          title=None
    ),
    alt.Color("nakladatel:N", 
              scale=alt.Scale(range=['#DFBD6E', '#C8414D', '#6B8EAA', '#33444C','#588438'  ,'#F2E9DE']), ## '# ['#a7ddd6','#001f80','#e6e6e4','#e5db00','#e49996','#9cc002']), 
              title=None,
               sort=['Orbis','SPN','Železný','MOBA','Euromedia','jiné'],
              legend=alt.Legend(orient="top")
    )).properties(
    width=kredity['sirka'] * 1.2, height=kredity['vyska_nizkych'] * 1.5,autosize={'type': 'fit-x', 'contains': 'padding'}
).configure_view(stroke='transparent').configure_view(stroke='transparent').configure_axisX(grid=False, domain=False)

graf_nejvesi

In [ ]:
me_to_neurazi(graf_nejvesi, soubor="01_nejvetsi_nakladatelstvi", kredity=kredity['default'])